In [ ]:
# ========================
# 🚀 FINAL STABLE SERVER
# ========================

import os
import uvicorn
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import FileResponse
from pyngrok import ngrok
import shutil
import edge_tts
import nest_asyncio
import subprocess
import sys

# 1. ROBUST DOWNLOADER
def download_model():
    model_path = "Wav2Lip/checkpoints/wav2lip_gan.pth"

    # List of Mirrors (If one fails, it tries the next)
    mirrors = [
        "https://huggingface.co/camenduru/Wav2Lip/resolve/main/checkpoints/wav2lip_gan.pth",
        "https://github.com/JustinsGlib/Wav2Lip/releases/download/v.0.0.1/wav2lip_gan.pth"
    ]

    # Check if exists and is valid size (>100MB)
    if os.path.exists(model_path):
        if os.path.getsize(model_path) < 100 * 1024 * 1024: # < 100MB
            print("⚠️ Found broken file. Deleting...")
            os.remove(model_path)
        else:
            print("✅ Model already exists.")
            return

    print("⬇️ Downloading Model Weights...")
    for url in mirrors:
        try:
            print(f"   Trying mirror: {url}...")
            # Use curl because it handles redirects better than wget
            exit_code = os.system(f"curl -L -o {model_path} {url}")

            # Verify download
            if os.path.exists(model_path) and os.path.getsize(model_path) > 100 * 1024 * 1024:
                print("   ✅ Download Success!")
                return
        except Exception as e:
            print(f"   ❌ Mirror failed: {e}")

    raise Exception("All download mirrors failed! Check internet connection.")

# Run the download
if not os.path.exists("Wav2Lip/checkpoints"):
    os.makedirs("Wav2Lip/checkpoints", exist_ok=True)
download_model()

# 2. SETUP APP
nest_asyncio.apply()
app = FastAPI()

@app.get("/health")
def health():
    return {"status": "active", "gpu": "T4"}

@app.post("/generate")
async def generate_video(
    text: str = Form(...),
    voice: str = Form(...),
    image: UploadFile = File(...)
):
    print(f"\n--> 📥 Request: '{text[:15]}...'")

    os.makedirs("temp", exist_ok=True)
    img_path = os.path.abspath("temp/input.jpg")
    audio_path = os.path.abspath("temp/audio.mp3")
    output_path = os.path.abspath("temp/output.mp4")

    if os.path.exists(output_path): os.remove(output_path)

    with open(img_path, "wb") as f: shutil.copyfileobj(image.file, f)
    communicate = edge_tts.Communicate(text, voice)
    await communicate.save(audio_path)

    # RUN INFERENCE
    cmd = [
        sys.executable, "Wav2Lip/inference.py",
        "--checkpoint_path", "Wav2Lip/checkpoints/wav2lip_gan.pth",
        "--face", img_path,
        "--audio", audio_path,
        "--outfile", output_path,
        "--resize_factor", "1"
    ]

    try:
        subprocess.run(cmd, check=True, capture_output=True, text=True)
        print("   ✅ Inference Success!")
        return FileResponse(output_path)
    except subprocess.CalledProcessError as e:
        print("❌ AI ERROR:")
        print(e.stderr)
        return {"error": f"AI Failed: {e.stderr[:200]}"}

# ---------------------------------------------------------
# ⬇️ TOKEN
# ---------------------------------------------------------
print("\n" + "="*60)
print("🔐 NGROK AUTHENTICATION REQUIRED")
print("="*60)
print("1. Go to: https://dashboard.ngrok.com/get-started/your-authtoken")
print("2. Login (or sign up free).")
print("3. Copy the token that starts with '2Fj...' or similar.")
print("="*60)

# Ask user for input securely
NGROK_TOKEN = getpass.getpass("👉 Paste your Ngrok Authtoken here and press Enter: ").strip()

if not NGROK_TOKEN:
    print("❌ ERROR: You must provide a token to continue.")
else:
    # Authenticate and Start
    ngrok.kill()
    print("🔄 Authenticating...")
    !ngrok config add-authtoken {NGROK_TOKEN}

    try:
        public_url = ngrok.connect(8000).public_url
        print(f"\n🚀 API ONLINE: {public_url}")
        print("👉 Copy this URL to your App")

        config = uvicorn.Config(app, port=8000)
        server = uvicorn.Server(config)
        await server.serve()

    except Exception as e:
        print(f"❌ Connection Error: {e}")